In [ ]:
import requests
import re
import os

In [ ]:
# judge if the folder exist. 判断图集的文件夹是否存在，如果不存在则建立一个
def fJudgeFile(vSeqName):
    global vProtscalFoldName
    vProtscalFoldName = "E:\\Store\\code\\02.Python\\03.Request\\Bio" + "\\" + vSeqName + "\\Protscal疏水性分析\\"
    os.makedirs(vProtscalFoldName, exist_ok = True)
    os.chdir(vProtscalFoldName)
    os.makedirs("Pic", exist_ok = True)
    os.makedirs("Text", exist_ok = True)

In [ ]:
def fGetProtscal(vSeq):
    vUrl = "https://web.expasy.org/cgi-bin/protscale/protscale.pl?1"
    vPostData ={"sequence": vSeq,
                "scale": "Hphob. / Kyte & Doolittle",
                "window": "9"}
    r = requests.post(vUrl, data = vPostData)
    return(r.text)

In [ ]:
def fSaveProtscalPic(vHtml):
    global vCode
    vCode = str(re.search("/tmp/pscale(.*?)\.gif", vHtml, re.S).group(1))    
    vPicA = "https://web.expasy.org" + "/tmp/pscale" + vCode + ".gif"    
    vPicB = "https://web.expasy.org" + "/tmp/pscale" + vCode + ".ps"
    vTxtA = "https://web.expasy.org" + "/tmp/scores" + vCode + ".txt"
    vTxtB = "https://web.expasy.org" + "/tmp/scoresmin" + vCode + ".txt"
    with open("Pic\\pscale.gif", "wb") as vPic1:
        r = requests.get(vPicA)
        vPic1.write(r.content)
    with open("Pic\\pscale.ps", "wb") as vPic2:
        r = requests.get(vPicB)
        vPic2.write(r.content)
    with open("Text\\scores.txt", "wb") as vTxt1:
        r = requests.get(vTxtA)
        vTxt1.write(r.content)
    with open("Text\\scoresmin.txt", "wb") as vTxt2:
        r = requests.get(vTxtB)
        vTxt2.write(r.content)

In [ ]:
def fSoupProtscal(vHtml):
    vProtscal1 = re.search("(\<h1\>ProtScale[\s\S]*?)\<br clear=", vHtml, re.S).group(1)
    vProtscal2 = vProtscal1.replace("<h1>ProtScale", "<h1>ProtScale疏水性分析").replace("User-provided sequence:", "用户数据：")
    vProtscal3 = vProtscal2.replace("SEQUENCE LENGTH", "序列长度").replace("Using the scale", "使用的计量方法：")
    vProtscal4 = vProtscal3.replace("/protscale/pscale/Hphob.Doolittle.html", "https://web.expasy.org/protscale/pscale/Hphob.Doolittle.html")
    vProtscal5 = vProtscal4.replace(", the individual values", "  20种氨基酸的计算值：").replace("for the 20 amino acids are:", "")
    vProtscal6 = vProtscal5.replace("Weights for window positions", "显示位置的权重").replace("using", "")
    vProtscal7 = vProtscal6.replace("weight variation model", "线性质量差异模型(linearweight variation model)")
    vProtscal8 = vProtscal7.replace("The results of your ProtScale query are", "ProtScale提供以下格式的查询结果供使用：").replace("available in the following formats:", "")
    vProtscal9 = vProtscal8.replace("Image in GIF-format", "gif图片").replace("Image in Postscript-format", "Postscript格式的图片")
    vProtscal10 = vProtscal9.replace("Numerical format (verbose)", "详细格式的数据(txt)").replace("Numerical format (minimal, to be exported into an external application)", "最简格式的数据(txt)")
    vProtscal11 = vProtscal10.replace("/tmp/pscale" + vCode + ".gif", "file:///" + vProtscalFoldName + "\Pic" + "\pscale.gif")
    vProtscal12 = vProtscal11.replace("/tmp/pscale" + vCode + ".ps", "file:///" + vProtscalFoldName + "\Pic" + "\pscale.ps")
    vProtscal13 = vProtscal12.replace("/tmp/scores" + vCode + ".txt", "file:///" + vProtscalFoldName + "\Text" + "\scores.txt")
    vProtscal14 = vProtscal13.replace("/tmp/scoresmin" + vCode + ".txt", "file:///" + vProtscalFoldName + "\Text" + "\scoresmin.txt")
    vProtscal = vProtscal14
    with open("Protscal.html", "w") as f:
        f.write(vProtscal)

In [ ]:
def main(vSeqName, vSeq):
    print("=====开始Protscal疏水性分析=====")
    fJudgeFile(vSeqName)
    print("---------已创建存储目录---------")
    print("----------开始提交数据----------")
    vHtml = fGetProtscal(vSeq)
    print("---------已获取分析结果---------")
    print("----------开始保存数据----------")
    fSaveProtscalPic(vHtml)
    print("----------开始保存报告----------")
    fSoupProtscal(vHtml)
    print("---------结果已保存完毕---------")

In [ ]:
vSeqName = "Ldlr"
vSeq = "MRTGTGSRLMPLDFRKIPQPKCIYTKRQRRKKKMRQHGAELRGSQPAGAVSDGTARSRLFFINSSAHHIAMGECQDEFAIAQLPCKCCLLPIPLLHMNKFSSIQCHKGLDLPVICCRLIKLHPQMKADLCVLEGAVCFYDDSIPVLLDDCGRLGHASHLPGGKAHTE"
main(vSeqName, vSeq)